# Functional Programming Techniques in Python: Part 2

In [Part 1](https://github.com/mpkocher/Functional-Programming-Techniques-In-Python/blob/main/Functional-Python-Part-1.ipynb), we introduced core Functional Programming techniques in Python.

One of the design patterns/techniques is to leverage closures. This is demonstrated in Part 1.

An example commonly given is:

```python
def say(message):
    def f(name):
        return " ".join([message, name])
    return f

say_hello = say("Hello")
print(say_hello("Steve")
```
Or 

```python
def adder(n):
    def f(m):
        return n + m
    return f

add_two = adder(2)
x = add_two(9)
print(x)
```

There's often a jump from these hello-world examples to using closures as central design patterns in your application. 

In Part 2, we're going to use these functional techniques to build a client interface to a REST API. Hopefully this REST client will provide a concrete example to help bridge the gap from the hello-world examples.


## Leveraging Functions in Design

As an example, we would like to interact with REST interface and extract some data for our data science team and make a client API to data source. 

For this example, we'll using the test service at https://jsonplaceholder.typicode.com/. 

The general model can be captured as set of transforms:

- construct URL
- construct headers
- make request
- transform request to desired output format (and also handle errors)

In [1]:
import requests
import functools
from collections import namedtuple
import datetime
import logging
import sys
# This requires python >= 3.7
from dataclasses import dataclass

In [2]:
print("Today is {}".format(datetime.datetime.now()))

Today is 2019-02-15 03:22:13.500830


In [3]:
BASE_URL = "https://jsonplaceholder.typicode.com"

## Iteration #1 Make Request from Relative Segments of the URL

As a starting point, let's add a mechanism to make requests from relative segment of interest (e.g., `/todos/1`) instead of the full URL.

This can be accomplished by using a simple closure. 

In [4]:
def to_get(base_url):
    def f(segment, **kwgs):
        url = "/".join([base_url, segment])
        return requests.get(url)
    return f

In [5]:
rget = to_get(BASE_URL)

In [6]:
first_todo = rget("todos/1").json()

In [7]:
first_todo

{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}

In [8]:
print("Found {} todos".format(len(rget("todos").json())))

Found 200 todos


Nice. We've got a basic working model. We can successfully make requests to any endpoint and return JSON from the server. 

However, we need to be able to set default headers (for example auth token) as well as handle errors and transform the response to desired format. This will also enable us to remove the duplicate `.json` call on every response.

Let's extend the our original implementation to add headers. 

In [9]:
def to_get(base_url, headers=None):
    def f(segment, **kw):
        h = kw.get('headers', {})
        headers.update(h)
        url = "/".join([base_url, segment])
        kw['headers'] = headers
        print("Making request {} with headers:{}".format(url, headers))
        return requests.get(url, **kw)
    return f

In [10]:
DEFAULT_HEADER = {"x-my-header": "12345"}
rget = to_get(BASE_URL, DEFAULT_HEADER)

In [11]:
rget('todos/1').json()

Making request https://jsonplaceholder.typicode.com/todos/1 with headers:{'x-my-header': '12345'}


{'userId': 1, 'id': 1, 'title': 'delectus aut autem', 'completed': False}

## CheckPoint

Similar to the URL case, we added the default headers to the `to_get` 'factory-ish' closure model.

- Added getting requests from relative urls
- Added getting requests with default headers

## Iteration #2 Add Concrete Model


Now, let's address the transformation from `dict` to concrete models. 

As python programmers we want to judiciously use `dict`s and not get into dictionary-mania. Having concrete models will help use refactor our code as well as demonstrate the general pipeline transformation model. 

For this example, let's define a class using Python 3.7 [dataclass](https://docs.python.org/3/library/dataclasses.html) feature. This could also be done using named tuples in 2.7 or using the [attrs](https://www.attrs.org/en/stable/) lib.

In [12]:
from dataclasses import dataclass, fields

# https://stackoverflow.com/questions/51736938/python-3-7-how-to-validate-typing-attributes
# why is this not in the stdlib?
def validate(instance):
    for field in fields(instance):
        attr = getattr(instance, field.name)
        if not isinstance(attr, field.type):
            msg = "Field {0.name} (value='{2}') is of type {1}, should be {0.type}".format(field, type(attr), attr)
            raise TypeError(msg)


@dataclass
class Todo:
    id: int
    user_id: int
    title: str
    completed: bool
        
    def __post_init__(self):
        validate(self)
        
    @staticmethod
    def from_d(d):
        # Note, we've decoupled the response field names of the server from our data models.
        # this renaming may or may not be a useful layer of abstraction for your use case
        return Todo(d['id'], d['userId'], d['title'], d['completed'])

In [13]:
Todo.from_d(rget('todos/1').json())

Making request https://jsonplaceholder.typicode.com/todos/1 with headers:{'x-my-header': '12345'}


Todo(id=1, user_id=1, title='delectus aut autem', completed=False)

Great. We've got our model being returned. 

Now let's wire an error handler as a general transform of `(response) -> (response)` and build up a general pipeline to process the response.

In [14]:
def default_error_handler(response):
    if not response.ok:
        # This can also be done via .raise_for_status()
        raise Exception("Failed request to {} status:{} {}".format(response.url, response.status_code, response.content[:25]))
    return response

In [15]:
def to_todo(response):
    return Todo.from_d(response.json())

Now let's compose the error handling and the transformation.

Let's define a general `compose` function. 

In [16]:
def compose(f, g):
    # this will return f(g(x))
    def func(*args, **kw):
        return f(g(*args, **kw))
    return func

In [17]:
# this has a type signtuare of (response) -> Todo
handle_error_to_todo = compose(to_todo, default_error_handler)

In [18]:
handle_error_to_todo(rget('todos/1'))

Making request https://jsonplaceholder.typicode.com/todos/1 with headers:{'x-my-header': '12345'}


Todo(id=1, user_id=1, title='delectus aut autem', completed=False)

## Iteration #3 Wire the Transforms into Response Processing

Nice. We've got all of core functionality in nice small pieces. 

Let's take another pass at our core `to_get` interface by adding the `transform` as a first class citizen. 

In [19]:
def null_tranform(response):
    return response

def to_get(base_url, headers=None):
    def f(segment, **kw):
        url = "/".join([base_url, segment])
        
        h = kw.get('headers', {})
        headers.update(h)
        kw['headers'] = headers
        
        transform_func = kw.pop('transform', null_tranform)
        
        print("Making request {} with headers:{} with transform {}".format(url, headers, transform_func))
        return transform_func(requests.get(url, **kw))
    return f

In [20]:
rget = to_get(BASE_URL, DEFAULT_HEADER)

In [21]:
rget('todos/1', transform=handle_error_to_todo)

Making request https://jsonplaceholder.typicode.com/todos/1 with headers:{'x-my-header': '12345'} with transform <function compose.<locals>.func at 0x1040e6488>


Todo(id=1, user_id=1, title='delectus aut autem', completed=False)

In [22]:
# Now trigger an error
rget('does-not-exist', transform=handle_error_to_todo)

Making request https://jsonplaceholder.typicode.com/does-not-exist with headers:{'x-my-header': '12345'} with transform <function compose.<locals>.func at 0x1040e6488>


Exception: Failed request to https://jsonplaceholder.typicode.com/does-not-exist status:404 b'{}'

## Iteration #4 Improve Error Handling

Note that we could also add default error handling to `to_get(base_url, headers=None, error_handler=None)` to avoid having to wire this into every transform. 

Let's wire that in. 

In [23]:
def to_get(base_url, headers=None, error_handler=None):
    def f(segment, **kw):
        url = "/".join([base_url, segment])
        
        h = kw.get('headers', {})
        headers.update(h)
        kw['headers'] = headers
        
        transform_func = kw.pop('transform', null_tranform)
        err_handler = null_tranform if error_handler is None else error_handler
        transform = compose(transform_func, err_handler)
        
        print("Making request {} with headers:{} with transform {}".format(url, headers, transform))
        return transform(requests.get(url, **kw))
    return f

In [24]:
rget = to_get(BASE_URL, headers=DEFAULT_HEADER, error_handler=default_error_handler)

In [25]:
rget('todos/1', transform=to_todo)

Making request https://jsonplaceholder.typicode.com/todos/1 with headers:{'x-my-header': '12345'} with transform <function compose.<locals>.func at 0x1041ddea0>


Todo(id=1, user_id=1, title='delectus aut autem', completed=False)

Nice. Everything still works as expected. 

Let's add a tranform for the `Todos` which will be `list[Todo]`

In [26]:
def to_todos(response):
    return [Todo.from_d(x) for x in response.json()]

In [27]:
todos = rget('todos', transform=to_todos)
print("Found {} todos".format(len(todos)))
todos[-1]

Making request https://jsonplaceholder.typicode.com/todos with headers:{'x-my-header': '12345'} with transform <function compose.<locals>.func at 0x1041dd9d8>
Found 200 todos


Todo(id=200, user_id=10, title='ipsam aperiam voluptates qui', completed=False)

## Check Point

With a handful of functions were able to have a customizable and extendible layer to fetch requests, validate data from the server and transform into the desired output format. **The surface area of extensibility of this approach is very large**.

- Added getting response from relative segment of the URL
- Added setting custom headers
- Add custom error handler
- Add custom tranformation from json/dict to concrete data model using `dataclasses`

Let's fix one last issue. The hardcoded `print` call is a quite terrible. Let's configure the logging to follow the same transformation pattern by leveraging the general response pipeline transform mechanism in the request. 

First, let's extend the original compose to take a list of funcs to compose to help build up our pipeline.

In [28]:
def compose(*funcs):
    """Functional composition
    [f, g, h] will be f(g(h(x)))
    """
    def compose_two(f, g):
        def c(x):
            return f(g(x))
        return c
    return functools.reduce(compose_two, funcs)

Now let's wire in the logger. 

In [29]:
def to_get(base_url, headers=None, error_handler=None, logger=None):
    def f(segment, **kw):
        url = "/".join([base_url, segment])
        
        h = kw.get('headers', {})
        headers.update(h)
        kw['headers'] = headers
        
        transform_func = kw.pop('transform', null_tranform)
        err_handler = null_tranform if error_handler is None else error_handler
        logger_func = null_tranform if logger is None else logger
        
        # Building custom response pipeline processor
        transform = compose(transform_func, err_handler, logger_func)

        return transform(requests.get(url, **kw))
    return f

Let's define a few loggers to use for testing purposes.

In [30]:
def null_logger(response):
    return response

def simple_logger(response):
    # the timestamp is a bit odd
    now = datetime.datetime.now()
    elapsed = response.elapsed.total_seconds()
    print("{} URL:{} response:{} in {} sec".format(now.isoformat(), response.url, response.status_code, elapsed))
    return response

In [31]:
rget = to_get(BASE_URL, headers=DEFAULT_HEADER, error_handler=default_error_handler, logger=simple_logger)

In [32]:
rget('todos/1', transform=to_todo)

2019-02-15T03:22:44.295066 URL:https://jsonplaceholder.typicode.com/todos/1 response:200 in 0.067777 sec


Todo(id=1, user_id=1, title='delectus aut autem', completed=False)

In [33]:
rget = to_get(BASE_URL, headers=DEFAULT_HEADER, error_handler=default_error_handler)

In [34]:
# As expected, no logging
rget('todos/1', transform=to_todo)

Todo(id=1, user_id=1, title='delectus aut autem', completed=False)

## CheckPoint

Nice. We've got the logging sorted out and we've made the system more extensible. 

However, there's still a large issue to address. Consumers of the client API don't want to call `rget('todos/1', transform=to_todo)` to get a Todo from the service. This isn't a reasonable high level interface. It would more useful for custom cases, or perhaps debugging.

Let's bridge the function that we've built up with a sugar layer to provide an improved interface. This sugar layer will call down into our `rget` layer. We'll keep `rget` public to enable custom use cases. 

In [35]:
class TodoClient(object):
    
    DEFAULT_HEADER = DEFAULT_HEADER
    
    def __init__(self, base_url, headers=None, logger=simple_logger, error_handler=default_error_handler):
        # making this public so one-off cases can leverage the rget model. This is dipping
        # down into the internals a bit. 
        h = TodoClient.DEFAULT_HEADER if headers is None else headers
        self.rget = to_get(base_url, headers=h, error_handler=error_handler, logger=logger)
        
    def __repr__(self):
        return "<TodoClient {} >".format(self.rget)
    
    def get_todos(self):
        return self.rget('todos', transform=to_todos)
    
    def get_todo_by_id(self, ix):
        return self.rget('todos/{}'.format(ix), transform=to_todo)

In [36]:
client = TodoClient(BASE_URL, logger=null_logger)
client

<TodoClient <function to_get.<locals>.f at 0x1041ddd90> >

In [37]:
todos = client.get_todos()

In [38]:
len(todos)

200

In [39]:
client.get_todo_by_id(2)

Todo(id=2, user_id=1, title='quis ut nam facilis et officia qui', completed=False)

Let's wrap up two final issues here. 

1. How to extend the error handling to support a `dict` style `dict.get` approach and return `None` instead of raising exceptions on 404s when getting resources by an id.
2. We've got a bunch of small funcs and a client class. What's the best model to package up the code and decided what to put in `__all__`?


The first item is will require some changes to the core `default_error_handler`. Specifically, to not wrap the error in the general `Exception`. After this is wired in, then the next step would be to modifiy the transform that converts the response to a concrete data model. 

A crude and terse example is demonstrated below. 

In [40]:
def to_todo_or_none(response):
    try:
        return Todo.from_d(response.json())
    except (KeyError, AttributeError):
        # this is extremely bad form. Don't do this.
        # this should only catch the 404 case.
        return None

In [41]:
class TodoClient(object):
    
    DEFAULT_HEADER = DEFAULT_HEADER
    
    def __init__(self, base_url, headers=None, logger=simple_logger, error_handler=default_error_handler):
        self.base_url = base_url
        self.headers = TodoClient.DEFAULT_HEADER if headers is None else headers
        self.logger = logger
        self.error_handler = error_handler
        self.rget = to_get(base_url, headers=self.headers, error_handler=error_handler, logger=logger)
        
    def __repr__(self):
        return "<TodoClient {} >".format(self.rget)
    
    def get_todos(self):
        return self.rget('todos', transform=to_todos)
    
    def get_todo_by_id(self, ix):
        rg = to_get(self.base_url, headers=self.headers, logger=simple_logger, error_handler=None)
        return rg('todos/{}'.format(ix), transform=to_todo_or_none)

In [42]:
client = TodoClient(BASE_URL)

In [43]:
client.get_todo_by_id(1)

2019-02-15T03:22:51.900114 URL:https://jsonplaceholder.typicode.com/todos/1 response:200 in 0.072748 sec


Todo(id=1, user_id=1, title='delectus aut autem', completed=False)

In [44]:
bad_todo = client.get_todo_by_id("DOES-NOT-EXIST")

2019-02-15T03:22:52.616999 URL:https://jsonplaceholder.typicode.com/todos/DOES-NOT-EXIST response:404 in 0.244881 sec


In [45]:
bad_todo is None

True

This appears to work as expected. 

The customization requires going back to the `to_get` layer to generate a custom `rget` for this specific usecase. If this is a common enough case, then having a private method of `._to_get` might be useful.

The last issue is to decide what we will expose in our Python module to users via `__all__`. One possible approach is to bundle the util funcs as class constants with a class. This should make the code easy to navigate and digest. 

For example:

In [46]:
class Loggers(object):
    DEFAULT = simple_logger
    NULL = null_logger
    
class ErrorHandlers(object):
    DEFAULT = default_error_handler
    NULL = null_tranform

Then define `__all__` as `['TodoClient', 'to_rget', 'Loggers', 'ErrorHandlers']`.

This provides an minimal public interface for users. The `to_get` function should probably be heavily documented to help consumers understand how to extend the API. 

# Summary And Final Thoughts

We've built a client using a functional approach yielding a client that useful for hight level consumers as well as added extension points for custom usecases. The core `to_get` function can also be used in clients to other REST APIs. 

In Part 3, we'll look at building commandline tools in Python using argparse using our Functional Python toolbox.